In [1]:
import micromagneticmodel as mm
import oommfc as mc

import discretisedfield as df  # df is here chosen to be an alias for discretisedfield

p1 = (0, 0, 0)
p2 = (100e-9, 50e-9, 20e-9)

region = df.Region(p1=p1, p2=p2)

In [2]:
import pyvista as pv

# pv.set_jupyter_backend("trame")

In [3]:
region.pyvista()

Widget(value="<iframe src='http://localhost:33117/index.html?ui=P_0x7f0a245115b0_0&reconnect=auto' style='widt…

In [4]:
lx, ly, lz = 100e-9, 50e-9, 20e-9

subregions = {
    "bottom_subregion": df.Region(p1=(20e-9, 0, 0), p2=(40e-9, 50e-9, 10e-9)),
    "top_subregion": df.Region(p1=(80e-9, 40e-9, lz / 2), p2=(lx, ly, lz)),
}

cell = (5e-9, 5e-9, 5e-9)

region = df.Region(p1=(0, 0, 0), p2=(lx, ly, lz))
mesh = df.Mesh(region=region, cell=cell, subregions=subregions)

In [5]:
mesh.pyvista()

Widget(value="<iframe src='http://localhost:33117/index.html?ui=P_0x7f099d17a070_1&reconnect=auto' style='widt…

In [6]:
mesh.pyvista.subregions()

Widget(value="<iframe src='http://localhost:33117/index.html?ui=P_0x7f099d1bc7f0_2&reconnect=auto' style='widt…

In [7]:
a, b, c = 5e-9, 3e-9, 2e-9
cell = (0.5e-9, 0.5e-9, 0.5e-9)

mesh = df.Mesh(p1=(-a, -b, -c), p2=(a, b, c), cell=cell)


def norm_fun(pos):
    x, y, z = pos
    if (x / a) ** 2 + (y / b) ** 2 + (z / c) ** 2 <= 1:
        return 1e6
    else:
        return 0


def value_fun(pos):
    x, y, z = pos
    c = 1e9
    return (-c * y, c * x, c * z)


field = df.Field(mesh, nvdim=3, value=value_fun, norm=norm_fun, valid="norm")

In [8]:
field.pyvista(cmap="viridis", scalars="x")

Widget(value="<iframe src='http://localhost:33117/index.html?ui=P_0x7f099d17a970_3&reconnect=auto' style='widt…

In [9]:
field.pyvista(cmap="viridis", scalars="x")

Widget(value="<iframe src='http://localhost:33117/index.html?ui=P_0x7f0997f91940_4&reconnect=auto' style='widt…

In [10]:
field.pyvista.contour(isosurfaces=5)

Widget(value="<iframe src='http://localhost:33117/index.html?ui=P_0x7f0997f91880_5&reconnect=auto' style='widt…

In [11]:
field.pyvista.contour(isosurfaces=[-1e5, 0, 1e5])

Widget(value="<iframe src='http://localhost:33117/index.html?ui=P_0x7f0997f75ee0_6&reconnect=auto' style='widt…

In [12]:
field.pyvista.streamlines()

Widget(value="<iframe src='http://localhost:33117/index.html?ui=P_0x7f0997f91580_7&reconnect=auto' style='widt…

In [13]:
field.x.pyvista.scalar()

Widget(value="<iframe src='http://localhost:33117/index.html?ui=P_0x7f0997f75b80_8&reconnect=auto' style='widt…

In [14]:
field.pyvista.valid()

Widget(value="<iframe src='http://localhost:33117/index.html?ui=P_0x7f0997f29b80_9&reconnect=auto' style='widt…

In [15]:
plotter = pv.Plotter()
field.pyvista(plotter=plotter, cmap="viridis")
field.pyvista.contour(plotter=plotter, isosurfaces=10, scalars="x")
plotter.show()

Widget(value="<iframe src='http://localhost:33117/index.html?ui=P_0x7f0997ef2640_10&reconnect=auto' style='wid…